In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
                    sep="|",
                    inferSchema=True,
                    header=True,
                    quote="'").limit(10)
display(df)

# Array Type

## Create Array Type column

In [0]:
from pyspark.sql.functions import split

result_df = (
    df.withColumn("skills", split("col_skills", ","))
    .withColumn(
        "current_expected_salary",
        split("col_current_expected_salary", ",").cast("array<int>"),
    )
    .drop("col_skills", "col_current_expected_salary")
)
result_df.printSchema()
result_df.display()

# Accessing Array Column

In [0]:
from pyspark.sql.functions import col
result_df.select(col("skills"), col("skills").getItem(0), col("skills")[0]).display()


# problem statement
> Derive 2 new coloumns current_salary and expected_salary from current_expected_salary where 
1. First elelement is current salary
2. Second element is expected salary

In [0]:
from pyspark.sql.functions import col

result_df.select(
    col("current_expected_salary"),
    col("current_expected_salary")[0].alias("current_salary"),
    col("current_expected_salary")[1].alias("Expected_Salary"),
).display()

# Problem
Fetch employee names whose current_salary >  expected_salary

In [0]:
# result_df.filter(col("current_salary") > col("Expected_Salary"))

from pyspark.sql.functions import col

result_df.select(
    col("current_expected_salary"),
    col("current_expected_salary")[0].alias("current_salary"),
    col("current_expected_salary")[1].alias("Expected_Salary"),
).filter(
    col("current_expected_salary")[0] > col("current_expected_salary")[1]
).display()

# result_df.filter(col("current_expected_salary")[0] > col("current_expected_salary")[1]).display()

In [0]:
from pyspark.sql.functions import size, array_distinct, array_contains

result_df.select(
    col("name"),
    col("skills"),
    size("skills"),
    array_contains("skills", "PySpark"),
    array_distinct("skills").alias("distinct_skills"),
).display()



## Fetch all employee who has skill pyspark

In [0]:
from pyspark.sql.functions import array_contains

result_df.select(
    col("name"),
 ).filter(array_contains("skills", "PySpark")).display()

## Increase expected salary by 30% if skill as pyspark

In [0]:
from pyspark.sql.functions import array_contains, when, col

result_df.withColumn(
    "base_salary",
    when(
        array_contains("skills", "PySpark"), (col("current_expected_salary")[1] * 1.3).cast("decimal(18,2)")
    ).otherwise(col("current_expected_salary")[1]),
).display()

In [0]:
from pyspark.sql.functions import explode
#result_df.select("skills", explode("skills")).display()
result_df.select(explode("skills").alias("words")).groupBy("words").count().display()
 

# Struct Type

In [0]:
df = spark.read.json(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/product_Information_001.json",
    multiLine=True,
)
df.printSchema()
#display(df)

In [0]:
from pyspark.sql.functions import col

df.select(
    col("product_id"),
    col("name"),
    col("details.screen.size").alias("screen_size"),
    col("details.memory.size").alias("memory_size"),
    col("details.storage.capacity").alias("storage_capacity"),
).display()

##